In [1]:
import pandas as pd

## Load Revere Relationships

**N.B.** The timestamps in the Revere data appear to be timezone specific. When converted to datetime values, these will be in GMT thus any non-timezone times e.g., CRSP data, will need to be shifted to GMT before merging

In [45]:
date_cols = ['start_', 'end_']
other_cols = ['supplier_id', 'customer_id']
cols_to_keep = date_cols + other_cols
rev = pd.read_csv('/mnt/ext_stor/Revere/supply_chain.csv', usecols=cols_to_keep)

# If a relationship hasn't expired then it has date 
# 'Fri Dec 31 19:00:00 EST 3999' which is out of range.
rev['end_'] = rev.end_.replace('Fri Dec 31 19:00:00 EST 3999', 'Fri Dec 31 19:00:00 EST 2100')

# Convert date columns to datetime dtype
for date_col in date_cols:
    rev[date_col] = pd.to_datetime(rev[date_col])
    
print('Loaded {} relationships'.format(len(rev)))
print('Data has {} unique suppliers and {} unique customers'.format(
    len(rev.supplier_id.unique()), len(rev.customer_id.unique())))

Loaded 1209986 relationships
Data has 70503 unique suppliers and 107925 unique customers


In [46]:
rev.head()

,start_,end_,supplier_id,customer_id
0,2003-04-03,2004-01-25,9840,12692
1,2003-04-03,2006-07-19,1090,4969
2,2003-04-03,2003-04-13,1399442,2939
3,2003-04-03,2004-01-11,8853,7384
4,2003-04-03,2005-05-04,180,5997


**N.B.** For any given period of time, it appears possible for there to be duplicate relationships between firms.

<img src="amd_sc_graph.png">

In [110]:
sel = [sup == 2409 and cust == 194 for sup, cust in zip(rev.supplier_id, rev.customer_id)]
rev.loc[sel]

,start_,end_,supplier_id,customer_id
146568,2006-06-02,2007-03-28,2409,194
169208,2007-03-28,2008-04-23,2409,194
196272,2008-04-23,2009-05-05,2409,194
218174,2009-05-05,2010-03-04,2409,194
235012,2010-03-04,2011-04-16,2409,194
1077333,2018-03-21,2018-03-21,2409,194
1077662,2011-04-16,2018-03-21,2409,194


In [47]:
rev.to_csv('/mnt/ext_stor/Revere/rels_start_end.csv')

## Load Company Meta Data

In [69]:
date_cols = ['start_', 'end_']
other_cols = ['id', 'name', 'ticker', 'cusip', 'isin', 'investor_contact_name']
cols_to_keep = date_cols + other_cols
rev_comp = pd.read_csv('/mnt/ext_stor/Revere/company_common.csv', usecols=cols_to_keep)

# If a company record hasn't expired then it has date 
# 'Fri Dec 31 19:00:00 EST 3999' which is out of range.
rev_comp['end_'] = rev_comp.end_.replace('Fri Dec 31 19:00:00 EST 3999', 'Fri Dec 31 19:00:00 EST 2100')

# Convert date columns to datetime dtype
for date_col in date_cols:
    rev_comp[date_col] = pd.to_datetime(rev_comp[date_col])
    
print('Loaded {} company records'.format(len(rev_comp)))
print('Data has {} unique companies'.format(len(rev_comp.id.unique())))

Loaded 2253482 company records
Data has 329347 unique companies


In [70]:
rev_comp.head()

,start_,end_,id,name,ticker,investor_contact_name,cusip,isin
0,2003-04-03,2005-08-05,1647,"Buckeye Partners, L.P.",BPL,Thomas B. Dornblaser,11823010,US1182301010
1,2003-04-03,2005-01-23,4084,FirstMerit Corporation,FMER,NaN,33791510,US3379151026
2,2003-04-03,2004-04-16,236891,Electrolux AB (ADR),ELUX,NaN,01019820,US0101982082
3,2003-04-03,2004-05-19,6729,Mpower Holding Corp.,MPOW,NaN,62473L30,US62473L3096
4,2003-04-03,2006-02-25,5101,Host Marriott Corporation,HMT,Greg Larson,44107P10,US44107P1049


In [103]:
rev_comp.loc[rev_comp.ticker == 'RMCF']

,start_,end_,id,name,ticker,investor_contact_name,cusip,isin
7081,2003-04-03,2003-11-13,8843,Rocky Mountain Chocolate Facto,RMCF,Franklin Crail,77467840,US7746784039
27195,2003-11-13,2006-07-08,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Franklin Crail,77467840,US7746784039
56002,2006-07-08,2006-07-25,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Virginia M Perez,77467840,US7746784039
56935,2006-07-25,2010-10-03,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Bryan J Merryman,77467840,US7746784039
414492,2010-10-03,2011-06-14,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Bryan J Merryman,77467840,US7746784039
590271,2011-06-14,2011-10-28,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Bryan J Merryman,77467840,US7746784039
666294,2011-10-28,2012-02-20,8843,"Rocky Mountain Chocolate Factory, Inc.",RMCF,Bryan J Merryman,77467840,US7746784039
744091,2012-02-20,2014-01-29,8843,Rocky Mountain Chocolate Factory,RMCF,Bryan J Merryman,77467840,US7746784039
830532,2014-01-29,2014-02-22,8843,Rocky Mountain Chocolate Factory,RMCF,Bryan J Merryman,77467840,US7746784039
957627,2014-02-22,2014-06-10,8843,Rocky Mountain Chocolate Factory,RMCF,Bryan J Merryman,77467840,US7746784039


In [93]:
rev_comp_recent = rev_comp.sort_values(['id', 'start_']).groupby(['id']).tail(1)

In [105]:
rev_comp_recent.loc[rev_comp_recent.id == 72594506]

,start_,end_,id,name,ticker,investor_contact_name,cusip,isin
2156437,2018-09-30,2101-01-01,72594506,ROCKY MOUNTAIN CHOCOLATE FACTORY INC /DE/,RMCF-US,NaN,77467X101,US77467X1019


In [96]:
rev_comp_recent.loc[rev_comp_recent.id == 6780]

,start_,end_,id,name,ticker,investor_contact_name,cusip,isin
2126545,2018-09-30,2101-01-01,6780,Microsoft Corp,MSFT-US,Bill Koefoed,594918104,US5949181045


In [102]:
rev_comp_recent.to_csv('/mnt/ext_stor/Revere/companies_recent.csv')

In [106]:
rev_comp_recent.loc[rev_comp_recent.ticker == 'AMD-US']

,start_,end_,id,name,ticker,investor_contact_name,cusip,isin
2204252,2018-10-02,2101-01-01,194,Advanced Micro Devices Inc,AMD-US,Michael Haase,007903107,US0079031078
